Instalamos la librería ktrain

In [ ]:
# install ktrain
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 128kB/s 
     |████████████████████████████████| 421.8MB 22kB/s 
     |████████████████████████████████| 6.7MB 45.3MB/s 
     |████████████████████████████████| 983kB 45.6MB/s 
     |████████████████████████████████| 245kB 53.6MB/s 
     |████████████████████████████████| 778kB 46.1MB/s 
     |████████████████████████████████| 471kB 49.5MB/s 
     |████████████████████████████████| 276kB 62.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 3.9MB 59.2MB/s 
     |████████████████████████████████| 450kB 50.1MB/s 
     |████████████████████████████████| 3.0MB 45.1MB/s 
     |████████████████████████████████| 1.1MB 52.1MB/s 
     |████████████████████████████████| 890kB 46.6MB/s 
  Created wheel for ktrain: filename=ktrain-0.19.4-cp36-none-any.whl size=25267674 sha256=bd4054b1dfc433aba50dcfca22db316cfc49997989e490164095e15cccd11f03
  Stored in directory: /root/.cache/pip/wheels/9a/16/56/5e5ba66

In [ ]:
# import ktrain
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

In [ ]:

import tensorflow as tf
import pandas as pd
datatrain = pd.read_csv('train.csv')

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train, x_val, y_train, y_val = train_test_split(datatrain['text'], datatrain['target'], shuffle=True, test_size = 0.2, random_state=42, stratify=datatrain['target'])

## Cargar y preprocesar el dataset

Usaremos la funcion texts_from_df que permite utilizar un dataframe de pandas para cargar los datos, utilizaremos como modo de preprocesamiento la opcion 'BERT'.

In [ ]:
x_train_b, y_train_b , preproc = text.texts_from_df(datatrain, "text", "target",   
                                                                         preprocess_mode='bert',
                                                                         maxlen = 80,
                                                                         max_features=35000 )

##  Cargar el modelo BERT precargado  

In [ ]:
model = text.text_classifier('bert', x_train_b, preproc=preproc)
learner = ktrain.get_learner(model,train_data=x_train_b, val_data=y_train_b, batch_size=6)

## Entrenamiento
Buscamos un learning rate adecuado y luego entrenamos el modelo.

In [ ]:
learner.lr_find()

In [ ]:
learner.lr_plot()

Se utiliza un learning rate de 2e-5 ya que es el mínimo en el gráfico de loss

In [ ]:
learner.autofit(2e-5)

In [ ]:
predictor=ktrain.get_predictor(learner.model, preproc)
datatest=pd.read_csv('test.csv')
datatest['target']=predictor.predict(datatest.text.tolist())

In [ ]:
datatest=datatest[['id', 'target']]
datatest

In [ ]:
datatest.to_csv('submission.csv', index=False)